In [24]:
import os
import numpy as np
import soundfile as sf
import glob
from scipy import signal
from scipy.signal import get_window
from librosa.filters import mel
from numpy.random import RandomState
import librosa

In [26]:
# downsampling for all wav files
rootDir = './chinesewavs'

dirName, subdirList, _ = next(os.walk(rootDir))

for subdir in sorted(subdirList):
    _,_, fileList = next(os.walk(os.path.join(dirName,subdir)))
    for fileName in sorted(fileList):
        filepath = os.path.join(rootDir, subdir, fileName)
        audio, sample_rate = librosa.load(filepath, sr=16000)
        output_file_path = filepath  
        sf.write(output_file_path, audio, sample_rate)
        

In [27]:
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a
    
    
def pySTFT(x, fft_length=1024, hop_length=256):
    
    x = np.pad(x, int(fft_length//2), mode='reflect')
    
    noverlap = fft_length - hop_length
    shape = x.shape[:-1]+((x.shape[-1]-noverlap)//hop_length, fft_length)
    strides = x.strides[:-1]+(hop_length*x.strides[-1], x.strides[-1])
    result = np.lib.stride_tricks.as_strided(x, shape=shape,
                                             strides=strides)
    
    fft_window = get_window('hann', fft_length, fftbins=True)
    result = np.fft.rfft(fft_window * result, n=fft_length).T
    
    return np.abs(result)    
    
def make_spect_fun(file):
    mel_basis = mel(16000, 1024, fmin=90, fmax=7600, n_mels=80).T
    min_level = np.exp(-100 / 20 * np.log(10))
    b, a = butter_highpass(30, 16000, order=5)
    x, fs = sf.read(file)

    # Remove drifting noise
    y = signal.filtfilt(b, a, x)
    # Add a little random noise for model robustness
    wav = y * 0.96
    # Compute spect
    D = pySTFT(wav).T
    # Convert to mel and normalize
    D_mel = np.dot(D, mel_basis)
    D_db = 20 * np.log10(np.maximum(min_level, D_mel)) - 16
    S = np.clip((D_db + 100) / 100, 0, 1)
    
    # save spect
    np.save(file[:-4],S.astype(np.float32), allow_pickle=False)

    return S.astype(np.float32)

In [28]:
if __name__ == '__main__':

    mel_basis = mel(sr=16000, n_fft=1024, fmin=90, fmax=7600, n_mels=80).T
    min_level = np.exp(-100 / 20 * np.log(10))
    b, a = butter_highpass(30, 16000, order=5)

    # audio file directory
    rootDir = './chinesewavs'
    # spectrogram directory
    targetDir = './chinesespmel'


    dirName, subdirList, _ = next(os.walk(rootDir))
    print('Found directory: %s' % dirName)

    for subdir in sorted(subdirList):
        print(subdir)
        if not os.path.exists(os.path.join(targetDir, subdir)):
            os.makedirs(os.path.join(targetDir, subdir))
        _,_, fileList = next(os.walk(os.path.join(dirName,subdir)))
        prng = RandomState(int(subdir[1:]))
        for fileName in sorted(fileList):
            mel_path = os.path.join(targetDir, subdir, fileName[:-4])
            if os.path.exists(mel_path+'.npy'):
                continue        
            # Read audio file
            x, fs = sf.read(os.path.join(dirName,subdir,fileName))
            # Remove drifting noise
            y = signal.filtfilt(b, a, x)
            # Add a little random noise for model robustness
            wav = y * 0.96 + (prng.rand(y.shape[0])-0.5)*1e-06
            # Compute spect
            D = pySTFT(wav).T
            # Convert to mel and normalize
            D_mel = np.dot(D, mel_basis)
            D_db = 20 * np.log10(np.maximum(min_level, D_mel)) - 16
            S = np.clip((D_db + 100) / 100, 0, 1)
            # save spect
            np.save(mel_path,S.astype(np.float32), allow_pickle=False)

Found directory: ./chinesewavs
S0010
S0017
S0018
S0019
S0038
S0041
